In [1]:

import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
from pmdarima.arima import ADFTest

In [2]:
sales = read_csv("../../Time_series_data.csv")

In [3]:
print(sales.shape[0])

105


In [4]:
x = sales['Month'][sales.shape[0]-1]
print(x)

2021-09


In [5]:
sales

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
...,...,...
100,2021-05,4618
101,2021-06,5312
102,2021-07,4298
103,2021-08,1413


In [6]:
sales['Month'] = pd.to_datetime(sales['Month'], errors = 'coerce')
sales.set_index('Month', inplace=True)

In [7]:
print(sales)

            Sales
Month            
2013-01-01   2815
2013-02-01   2672
2013-03-01   2755
2013-04-01   2721
2013-05-01   2946
...           ...
2021-05-01   4618
2021-06-01   5312
2021-07-01   4298
2021-08-01   1413
2021-09-01   5877

[105 rows x 1 columns]


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sales, test_size=0.2, shuffle=False)

arima_model = auto_arima(train, start_p=0, d=1, start_q=0,
                        max_p=5, max_d=5, max_q=5, start_P=0,
                        D=1, start_Q=0, max_P=5, max_D=5,
                        max_Q=5, m=12, seasonal=True,
                        error_action='warn', trace=True,
                        supress_warnings=True, stepwise=True,
                        random_state=20, n_fits=50)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1183.693, Time=0.08 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=1173.736, Time=0.25 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=1157.042, Time=0.56 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=1155.109, Time=0.19 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=1157.009, Time=0.49 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=1158.348, Time=1.21 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=1155.379, Time=0.29 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=1155.138, Time=0.27 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=1173.612, Time=0.07 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=1155.790, Time=0.43 sec
 ARIMA(0,1,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.21 sec

Best model:  ARIMA(0,1,1)(0,1,0)[12]          
Total fit time: 4.052 seconds


In [9]:
prediction = pd.DataFrame(
    arima_model.predict(n_periods=30),index=test.index )
prediction.columns = ['predicted_sales']
prediction.reset_index(inplace=True)

In [10]:
print(prediction)

        Month  predicted_sales
0  2020-01-01      2260.640695
1  2020-02-01      2520.640695
2  2020-03-01      2991.640695
3  2020-04-01      3361.640695
4  2020-05-01      2548.640695
5  2020-06-01      3607.640695
6  2020-07-01      3838.640695
7  2020-08-01      1359.640695
8  2020-09-01      4842.640695
9  2020-10-01      6045.640695
10 2020-11-01      9463.640695
11 2020-12-01     12697.640695
12 2021-01-01      1882.281390
13 2021-02-01      2142.281390
14 2021-03-01      2613.281390
15 2021-04-01      2983.281390
16 2021-05-01      2170.281390
17 2021-06-01      3229.281390
18 2021-07-01      3460.281390
19 2021-08-01       981.281390
20 2021-09-01      4464.281390


In [11]:
index_future_dates = pd.date_range(
    start=x, end='2023-01-1', freq='MS')

In [12]:
prediction_arima = pd.DataFrame(
    arima_model.predict(n_periods=36), index = index_future_dates)
prediction_arima.columns = ['predicted_sales']  
print(prediction_arima)

            predicted_sales
2021-09-01      4464.281390
2021-10-01      5667.281390
2021-11-01      9085.281390
2021-12-01     12319.281390
2022-01-01      1503.922084
2022-02-01      1763.922084
2022-03-01      2234.922084
2022-04-01      2604.922084
2022-05-01      1791.922084
2022-06-01      2850.922084
2022-07-01      3081.922084
2022-08-01       602.922084
2022-09-01      4085.922084
2022-10-01      5288.922084
2022-11-01      8706.922084
2022-12-01     11940.922084
2023-01-01              NaN
